In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,9)
plt.style.use('bmh')
from datetime import timedelta
from keras.models import *
from keras.layers import *
from sklearn.preprocessing import *
import seaborn as sns

In [2]:
# Load Dataset
def preprocess(file):
    data = pd.read_csv(file)
    HUElec = pd.melt(data, id_vars=["Date"], var_name='time')
    HUElec['time'] = pd.to_timedelta(HUElec['time'].apply(lambda t: f"{t[:5]}:00")) - timedelta(minutes=30)
    HUElec['Date'] = pd.to_datetime(HUElec['Date'], format="%m/%d/%Y")
    HUElec['Datetime'] = HUElec['Date'] + HUElec['time']
    HUElec = HUElec.set_index('Datetime')
    HUElec = HUElec.drop(['Date', 'time'], axis=1).sort_index()

    return HUElec

In [3]:
file = 'GTWYGas.csv'

data = preprocess(file)

In [4]:
df = data.resample(rule='d').sum()

df.head()

,value
Datetime,
2008-05-07,164.847
2008-05-08,282.808
2008-05-09,273.726
2008-05-10,182.219
2008-05-11,182.238


In [5]:
df['value_next_day'] = df['value'].shift(-1)

In [6]:
df = df.dropna(how='any', axis=0)

df.tail()

,value,value_next_day
Datetime,,
2022-02-23,4811.995,4354.241
2022-02-24,4354.241,4421.232
2022-02-25,4421.232,189.805
2022-02-26,189.805,200.970
2022-02-27,200.970,826.190


In [7]:
y_true = df['value']
y_pred = df['value_next_day']

In [9]:
from sklearn.metrics import *

print('RMSE: ', np.sqrt(mean_squared_error(y_true, y_pred)))
print('MSE: ', mean_squared_error(y_true, y_pred))
print('MAE: ', mean_absolute_error(y_true, y_pred))
print('MAPE: ', mean_absolute_percentage_error(y_true, y_pred)*100)
print('R2: ', r2_score(y_true, y_pred)*100)

RMSE:  1503.221384639588
MSE:  2259674.53123776
MAE:  714.6849563924678
MAPE:  3.614451765858643e+18
R2:  41.08765767321453
